# Retail Inventory Health Analysis

**Client:** Distressed Retail Client  
**Objective:** Reconcile three inventory data sources, identify actionable insights, and build a reusable analysis framework.

## Data Sources
| Source | Format | Description |
|--------|--------|-------------|
| POS System | CSV | 500K+ transactions with messy data |
| Inventory Mgmt | Excel | 265 products with manual overrides |
| E-commerce | JSON | 125K orders with different ID scheme |

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

# Load environment variables (for OpenAI API key)
load_dotenv()

# Display settings
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 50)
plt.style.use('seaborn-v0_8-whitegrid')

DATA_DIR = Path.cwd().parent / "data" / "raw"

## 1. Data Loading & Cleaning

Using the client-specific loader that handles:
- Multiple date formats in POS data
- SKU normalization across systems
- Notes column parsing for inventory overrides

In [ ]:
from clients.retail_client import RetailClientLoader

loader = RetailClientLoader(DATA_DIR)
data = loader.load_all()

pos = data.pos_transactions
inv = data.inventory
ecom = data.ecommerce_orders

print(f"POS Transactions: {len(pos):,} rows")
print(f"Inventory Items: {len(inv):,} rows")
print(f"E-commerce Orders: {len(ecom):,} rows")

## 2. Data Quality Assessment

Automated quality checks ran during loading. Let's review the findings.

In [ ]:
# Quality summary
for source, report in data.quality_reports.items():
    print(f"\n{'='*50}")
    print(f"{report.source_name}: {report.total_rows:,} rows")
    print(f"Critical issues: {len(report.critical_issues)}")
    print(f"Warnings: {len(report.warning_issues)}")
    
    for issue in report.issues[:5]:  # Show top 5
        print(f"  - [{issue.severity.upper()}] {issue.column}: {issue.description}")

### POS Data Issues

The POS system has significant data quality issues that the client should address:

In [ ]:
# Date format distribution (the messiness)
print("Sample of date formats found:")
print(pos['date'].head(20).tolist())

# Payment method inconsistencies
print("\nPayment method values (before cleaning):")
print(pos['payment_method'].value_counts())

# Missing data
print("\nMissing values:")
print(pos.isnull().sum()[pos.isnull().sum() > 0])

### Inventory Notes - Manual Overrides

The ops team uses the Notes column to track corrections. This is a workaround for system issues.

In [ ]:
# Show items with manual overrides
overrides = inv[inv['notes'].notna()][['item_code', 'description', 'qty_on_hand', 'physical_count_override', 'adjustment', 'notes']]
print(f"Items with notes: {len(overrides)}")
print(f"Items with physical count corrections: {inv['physical_count_override'].notna().sum()}")
print(f"Items with adjustments: {inv['adjustment'].notna().sum()}")

overrides.head(15)

## 3. Product Matching Across Systems

The three systems use different product identifiers:
- **POS:** Various SKU formats (SKU-XXXX, XXXX, XXXXA)
- **Inventory:** Numeric Item Codes
- **E-commerce:** ECOM-XXXXXX format

We'll match by normalized product name since IDs don't align.

In [ ]:
# Aggregate POS sales by normalized SKU
pos_sales = pos[pos['quantity'] > 0].copy()
pos_returns = pos[pos['quantity'] < 0].copy()

pos_by_sku = pos_sales.groupby('sku_normalized').agg(
    total_sold=('quantity', 'sum'),
    revenue=('line_total', 'sum'),
    transactions=('transaction_id', 'count'),
    product_name=('product_name', 'first'),
    last_sale=('date_parsed', 'max')
).reset_index()

# Add returns
returns_by_sku = pos_returns.groupby('sku_normalized')['quantity'].sum().abs().reset_index()
returns_by_sku.columns = ['sku_normalized', 'return_units']
pos_by_sku = pos_by_sku.merge(returns_by_sku, on='sku_normalized', how='left')
pos_by_sku['return_units'] = pos_by_sku['return_units'].fillna(0)

print(f"Unique products in POS: {len(pos_by_sku)}")
pos_by_sku.head(10)

In [ ]:
# Match POS to Inventory by normalized product name
from core.parsers import ProductNameNormalizer

normalizer = ProductNameNormalizer()

# Create lookup from inventory
inv_lookup = inv.set_index('description_normalized')[['item_code_normalized', 'qty_adjusted', 'reorder_level', 'retail_price', 'category', 'location']].to_dict('index')

# Match POS products to inventory
pos_by_sku['product_name_normalized'] = normalizer.normalize_series(pos_by_sku['product_name'])
pos_by_sku['inv_match'] = pos_by_sku['product_name_normalized'].map(
    lambda x: inv_lookup.get(x) if x in inv_lookup else None
)

matched = pos_by_sku[pos_by_sku['inv_match'].notna()]
unmatched = pos_by_sku[pos_by_sku['inv_match'].isna()]

print(f"Matched to inventory: {len(matched)} ({len(matched)/len(pos_by_sku)*100:.1f}%)")
print(f"Unmatched: {len(unmatched)} ({len(unmatched)/len(pos_by_sku)*100:.1f}%)")

## 4. Inventory Health Analysis

### 4.1 Stockout Risk

Products likely to run out based on current stock vs. sales velocity.

In [ ]:
from core.analysis import compute_sales_velocity, identify_stockout_risks

# Compute sales velocity
velocity = compute_sales_velocity(pos, sku_col='sku_normalized', qty_col='quantity', date_col='date_parsed')

# Identify stockout risks
stockout_risks = identify_stockout_risks(
    inv, velocity,
    inv_sku_col='item_code_normalized',
    inv_qty_col='qty_adjusted',
    vel_sku_col='sku_normalized'
)

print(f"Products at stockout risk: {len(stockout_risks)}")
print(f"  - Critical (≤7 days): {len(stockout_risks[stockout_risks['risk_level']=='critical'])}")
print(f"  - High (≤14 days): {len(stockout_risks[stockout_risks['risk_level']=='high'])}")
print(f"  - Medium (≤30 days): {len(stockout_risks[stockout_risks['risk_level']=='medium'])}")

In [ ]:
# Top stockout risks
stockout_display = stockout_risks[['item_code', 'description', 'qty_adjusted', 'avg_daily_sales', 'days_of_stock', 'risk_level', 'category']].head(15)
stockout_display

### 4.2 Dead Inventory

Products not moving that tie up capital.

In [ ]:
from core.analysis import identify_dead_inventory

dead_inv = identify_dead_inventory(
    inv, pos,
    inv_sku_col='item_code_normalized',
    inv_qty_col='qty_adjusted',
    inv_price_col='retail_price',
    txn_sku_col='sku_normalized',
    txn_date_col='date_parsed',
    dead_days_threshold=60
)

print(f"Dead inventory items (no sales in 60+ days): {len(dead_inv)}")
print(f"Total value at risk: ${dead_inv['value_at_risk'].sum():,.2f}")

In [ ]:
# Top dead inventory by value
dead_display = dead_inv[['item_code', 'description', 'qty_adjusted', 'retail_price', 'days_since_last_sale', 'value_at_risk', 'category']].head(15)
dead_display

### 4.3 Channel Comparison (In-Store vs Online)

In [ ]:
from core.analysis import compute_channel_comparison

channel_data = compute_channel_comparison(pos, ecom)

print("Channel Comparison:")
print("="*50)
print(f"{'Metric':<30} {'In-Store':>12} {'Online':>12}")
print("-"*50)
print(f"{'Revenue':.<30} ${channel_data['in_store']['total_revenue']:>10,.0f} ${channel_data['online']['total_revenue']:>10,.0f}")
print(f"{'Units Sold':.<30} {channel_data['in_store']['total_units']:>12,} {channel_data['online']['total_units']:>12,}")
print(f"{'Transactions':.<30} {channel_data['in_store']['transaction_count']:>12,} {channel_data['online']['transaction_count']:>12,}")
print(f"{'Avg Order Value':.<30} ${channel_data['in_store']['avg_order_value']:>10,.2f} ${channel_data['online']['avg_order_value']:>10,.2f}")
print(f"{'Return Rate':.<30} {channel_data['in_store']['return_rate']*100:>11.1f}% {channel_data['online']['return_rate']*100:>11.1f}%")
print("="*50)
print(f"In-store share of revenue: {channel_data['comparison']['revenue_split_instore_pct']:.1f}%")

In [ ]:
# Visualize channel comparison
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Revenue split
axes[0].pie(
    [channel_data['in_store']['total_revenue'], channel_data['online']['total_revenue']],
    labels=['In-Store', 'Online'],
    autopct='%1.1f%%',
    colors=['#2ecc71', '#3498db']
)
axes[0].set_title('Revenue Split')

# AOV comparison
channels = ['In-Store', 'Online']
aovs = [channel_data['in_store']['avg_order_value'], channel_data['online']['avg_order_value']]
axes[1].bar(channels, aovs, color=['#2ecc71', '#3498db'])
axes[1].set_ylabel('Average Order Value ($)')
axes[1].set_title('AOV Comparison')

# Return rates
return_rates = [channel_data['in_store']['return_rate']*100, channel_data['online']['return_rate']*100]
axes[2].bar(channels, return_rates, color=['#2ecc71', '#3498db'])
axes[2].set_ylabel('Return Rate (%)')
axes[2].set_title('Return Rate Comparison')

plt.tight_layout()
plt.savefig('../outputs/channel_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

### 4.4 Items Below Reorder Level

In [ ]:
below_reorder = inv[inv['below_reorder_level']].sort_values('qty_adjusted')
print(f"Items below reorder level: {len(below_reorder)}")

below_reorder[['item_code', 'description', 'qty_adjusted', 'reorder_level', 'category', 'location']].head(15)

## 5. Key Metrics Summary

In [ ]:
from core.analysis import compute_key_metrics

key_metrics = compute_key_metrics(inv, pos, ecom, stockout_risks, dead_inv)

print("Key Metrics:")
print("="*50)
for k, v in key_metrics.items():
    if isinstance(v, float) and v > 1000:
        print(f"{k}: ${v:,.2f}" if 'value' in k or 'revenue' in k else f"{k}: {v:,.2f}")
    elif isinstance(v, float):
        print(f"{k}: {v:.2f}")
    else:
        print(f"{k}: {v:,}" if isinstance(v, int) else f"{k}: {v}")

## 6. AI-Assisted Insight Generation

Using GPT-4 with structured outputs (Pydantic) to generate insights and recommendations.

**What the AI does well:**
- Synthesizing patterns across data points
- Generating natural language explanations
- Prioritizing recommendations

**What I verified/corrected:**
- All numeric values are computed programmatically, not by the LLM
- Rankings are cross-checked against the data
- Specific SKUs and product names are passed to the LLM, not generated

In [ ]:
import os

# Check if API key is available
if not os.getenv('OPENAI_API_KEY'):
    print("⚠️  OPENAI_API_KEY not set. Skipping AI insight generation.")
    print("   Set it in .env file to enable AI-generated insights.")
    ai_enabled = False
else:
    ai_enabled = True
    print("✓ OpenAI API key found")

In [ ]:
if ai_enabled:
    from core.insights import InsightGenerator
    
    # Prepare data for AI
    stockout_data = stockout_risks[['item_code', 'description', 'qty_adjusted', 'days_of_stock', 'risk_level', 'category']].head(20).to_dict('records')
    dead_inv_data = dead_inv[['item_code', 'description', 'qty_adjusted', 'days_since_last_sale', 'value_at_risk', 'category']].head(20).to_dict('records')
    
    quality_issues = []
    for source, report in data.quality_reports.items():
        for issue in report.issues:
            quality_issues.append({
                'source': source,
                'column': issue.column,
                'issue': issue.issue_type,
                'severity': issue.severity,
                'count': issue.count
            })
    
    # Empty reconciliation for now (would need more complex logic for full implementation)
    reconciliation_data = []
    
    # Generate insights
    generator = InsightGenerator(model="gpt-4o-mini")
    
    try:
        report = generator.generate_insights(
            stockout_data=stockout_data,
            dead_inventory_data=dead_inv_data,
            reconciliation_data=reconciliation_data,
            channel_data=channel_data,
            quality_issues=quality_issues,
            key_metrics=key_metrics
        )
        
        print("AI-Generated Executive Summary:")
        print("="*60)
        print(report.executive_summary)
    except Exception as e:
        print(f"Error generating AI insights: {e}")
        report = None

In [ ]:
if ai_enabled and report:
    print("\nAI-Generated Recommendations:")
    print("="*60)
    
    print("\n📦 Stockout Risks:")
    for risk in report.stockout_risks[:5]:
        print(f"  [{risk.risk_level.upper()}] {risk.product_name}: {risk.days_of_stock:.0f} days of stock")
        print(f"    → {risk.recommendation}")
    
    print("\n🏚️ Dead Inventory:")
    for dead in report.dead_inventory[:5]:
        print(f"  {dead.product_name}: ${dead.estimated_value:,.0f} value, {dead.days_since_last_sale} days")
        print(f"    → {dead.recommendation}")
    
    print("\n🔧 Data Quality Fixes:")
    for rec in report.data_quality_recommendations[:5]:
        print(f"  [{rec.priority.upper()}] {rec.system}: {rec.issue}")
        print(f"    Impact: {rec.business_impact}")
        print(f"    Fix: {rec.fix_recommendation}")

## 7. Data Quality Recommendations for Client

Based on our analysis, here are the issues the client should fix in their source systems:

In [ ]:
print("Data Quality Issues to Address:")
print("="*60)

print("\n1. POS SYSTEM")
print("-" * 40)
print("• Date formats: 4+ different formats across transactions")
print("  → Standardize on ISO format (YYYY-MM-DD)")
print(f"• Missing store_id: {pos['store_id'].isna().sum():,} transactions ({pos['store_id'].isna().sum()/len(pos)*100:.1f}%)")
print("  → Require store_id at point of sale")
print(f"• Missing customer_id: {pos['customer_id'].isna().sum():,} transactions")
print("  → Expected for cash sales, but review card transactions")
print("• Payment method inconsistency: 'CASH' vs 'Cash' vs 'cash'")
print("  → Enforce dropdown selection instead of free text")
print("• TEST/VOID transactions in production data")
print("  → Filter these at export or use separate test environment")

print("\n2. INVENTORY SYSTEM")
print("-" * 40)
print(f"• {inv['physical_count_override'].notna().sum()} items have manual count corrections")
print("  → Investigate why system counts are wrong")
print("  → Consider more frequent cycle counts")
print(f"• {inv['adjustment'].notna().sum()} items have manual adjustments")
print("  → These should flow through the system, not notes")

print("\n3. CROSS-SYSTEM")
print("-" * 40)
print("• No common product identifier across POS, Inventory, and E-commerce")
print("  → Implement a master product ID that all systems share")
print("• SKU formats vary: 'SKU-12345', '12345', '012345'")
print("  → Standardize SKU format with validation rules")

## 8. Save Outputs

In [ ]:
output_dir = Path.cwd().parent / "outputs"
output_dir.mkdir(exist_ok=True)

# Save stockout risks
stockout_risks[['item_code', 'description', 'qty_adjusted', 'avg_daily_sales', 'days_of_stock', 'risk_level', 'category', 'reorder_level']].to_csv(
    output_dir / 'stockout_risks.csv', index=False
)

# Save dead inventory
dead_inv[['item_code', 'description', 'qty_adjusted', 'retail_price', 'days_since_last_sale', 'value_at_risk', 'category']].to_csv(
    output_dir / 'dead_inventory.csv', index=False
)

# Save below reorder
below_reorder[['item_code', 'description', 'qty_adjusted', 'reorder_level', 'category', 'location']].to_csv(
    output_dir / 'below_reorder_level.csv', index=False
)

# Save key metrics
import json
with open(output_dir / 'key_metrics.json', 'w') as f:
    json.dump(key_metrics, f, indent=2)

print(f"Outputs saved to {output_dir}")
print("Files created:")
for f in output_dir.glob('*'):
    print(f"  - {f.name}")

---

## Summary of Findings

### Immediate Actions Required

1. **Reorder urgently**: Several items are critically low on stock (< 7 days supply)
2. **Review dead inventory**: Significant capital tied up in non-moving stock
3. **Fix POS date formats**: Data quality issue that makes analysis harder

### System Improvements Needed

1. Implement unified product ID across all systems
2. Move inventory adjustments from Notes to proper system fields
3. Standardize POS data entry (dates, payment methods)

### What's Reusable for Next Client

- `src/core/parsers.py`: Date and SKU parsers handle common retail formats
- `src/core/quality.py`: Data quality framework works with any pandas DataFrame
- `src/core/analysis.py`: Sales velocity, stockout risk, dead inventory logic
- `src/core/insights.py`: AI insight generator with Pydantic models

### What's Client-Specific

- `src/clients/retail_client.py`: Specific column mappings, Notes parsing patterns